In [ ]:
import os
import google.generativeai as genai
import pandas as pd
import json
from libraries.widget import build_ui

widgets_dict = build_ui()

In [ ]:
# --- Lấy giá trị từ widgets ---
input_file   = widgets_dict["input_drop"].value
prompt_file  = widgets_dict["prompt_drop"].value
article_col  = widgets_dict["article_drop"].value
summary_col  = widgets_dict["summary_drop"].value
start_row    = widgets_dict["start_data"].value
end_row      = widgets_dict["end_data"].value
api_key      = widgets_dict["API_drop"].value
model_name   = widgets_dict["model_drop"].value

# --- Xử lý tên file và đường dẫn ---
file_name    = os.path.splitext(input_file)[0]
input_path   = f"Unique/Evaluation/{file_name}.xlsx"
data         = pd.read_excel(input_path)
prompt       = prompt_file

# --- Xác định start và end row ---
start = int(start_row) if start_row and start_row.isdigit() and int(start_row) > 1 else 1
end   = int(end_row)   if end_row and end_row.isdigit() and int(end_row) > 1 else 1

# --- Xác định đường dẫn output ---
output_folder = f"Outputs/Evaluation/{file_name}"
if end == start:
    output_json = f"{output_folder}/{file_name} {summary_col} {start:05} {model_name}.json"
    output_xlsx = f"{output_folder}/{file_name} {summary_col} {start:05} {model_name}.xlsx"
else:
    output_json = f"{output_folder}/{file_name} {summary_col} {start:05}-{end:05} {model_name}.json"
    output_xlsx = f"{output_folder}/{file_name} {summary_col} {start:05}-{end:05} {model_name}.xlsx"

# --- Đặt tên cột article và summary ---
article_row = article_col if article_col else "article"
summary_row = summary_col if summary_col else "summary"
export_option = widgets_dict["export_drop"].value

GEMINI_API_KEY = api_key
SCORER_ID = model_name
genai.configure(api_key = GEMINI_API_KEY)

# --- Hiển thị thông tin ---
print(f"API key: {api_key}")
print(f"Model:   {model_name}")
print(f"Input:   {file_name}")
print(f"Article: {article_row}")
print(f"Summary: {summary_row}")
print(f"Output:  {file_name} {start:05} {model_name}" if end == start else f"Output:  {file_name} {start:05}-{end:05} {model_name}")

In [ ]:
from libraries.Processor import SummaryEvaluator

config = {
    "api_key": api_key,
    "model_name": model_name,
    "file_name": file_name,
    "input_path": input_path,
    "output_json": output_json,
    "output_xlsx": output_xlsx,
    "article_row": article_row,
    "summary_row": summary_row,
    "start": start,
    "end": end,
    "prompt": prompt,
}

In [ ]:
evaluator = SummaryEvaluator(config)
results = evaluator.run()
output_data = results

EXPORT


In [ ]:

output_dir = os.path.dirname(output_json)
os.makedirs(output_dir, exist_ok=True)

with open(output_json, "w", encoding="utf-8") as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4, separators=(",", ": "))

print(f"{output_json} saved successfully!")

In [ ]:
if export_option == "YES":
    with open(output_json, 'r', encoding='utf-8') as file:
        data = json.load(file)
    df = pd.json_normalize(data)
    df.to_excel(output_xlsx, index=False)
    print(f"{output_json} Reading...")
    print(f"{output_xlsx} saved successfully!")
else:
    print("Skip...")